In [1]:
import pandas as pd
from category_encoders import LeaveOneOutEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('car_price_prediction.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

В датасете относительно много данных и нет значений nan.
Непонятные колонки: Levy.
Стоит посмотреть колонки типа object на наличие "пропусков", бинарных или числовых типов.

In [4]:
df["Levy"].unique()

array(['1399', '1018', '-', '862', '446', '891', '761', '751', '394',
       '1053', '1055', '1079', '810', '2386', '1850', '531', '586',
       '1249', '2455', '583', '1537', '1288', '915', '1750', '707',
       '1077', '1486', '1091', '650', '382', '1436', '1194', '503',
       '1017', '1104', '639', '629', '919', '781', '530', '640', '765',
       '777', '779', '934', '769', '645', '1185', '1324', '830', '1187',
       '1111', '760', '642', '1604', '1095', '966', '473', '1138', '1811',
       '988', '917', '1156', '687', '11714', '836', '1347', '2866',
       '1646', '259', '609', '697', '585', '475', '690', '308', '1823',
       '1361', '1273', '924', '584', '2078', '831', '1172', '893', '1872',
       '1885', '1266', '447', '2148', '1730', '730', '289', '502', '333',
       '1325', '247', '879', '1342', '1327', '1598', '1514', '1058',
       '738', '1935', '481', '1522', '1282', '456', '880', '900', '798',
       '1277', '442', '1051', '790', '1292', '1047', '528', '1211',
       

Levy перевод - сбор. Похоже на какой-то налог.
Учитывая что всего 9 уникальных значений, то это дискретная величина.

In [5]:
df["Levy"].value_counts()

Levy
-       5819
765      486
891      461
639      410
640      405
        ... 
3156       1
2908       1
1279       1
1719       1
1901       1
Name: count, Length: 559, dtype: int64

In [6]:
df["Levy"] = df["Levy"].replace("-", 0)
df = df.astype({"Levy": "int32"})

По информации с kaggle: "Tax of importing and exporting the cars."
Так же указано, что не определена валюта и смысл для "-".

Принял "-", как 0 и изменил тип колонки на числовой.

In [7]:
df["Manufacturer"].value_counts()

Manufacturer
HYUNDAI          3769
TOYOTA           3662
MERCEDES-BENZ    2076
FORD             1111
CHEVROLET        1069
                 ... 
TESLA               1
PONTIAC             1
SATURN              1
ASTON MARTIN        1
GREATWALL           1
Name: count, Length: 65, dtype: int64

In [8]:
df[df["Manufacturer"] == "სხვა"]

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
2358,45779593,25089,0,სხვა,IVECO DAYLY,2007,Microbus,No,Diesel,2.3 Turbo,328000 km,4.0,Manual,Rear,04-May,Left wheel,White,1
4792,39223518,9408,0,სხვა,GONOW,2005,Jeep,Yes,Petrol,2.3,102000 km,4.0,Manual,Rear,04-May,Left wheel,Silver,2


Есть необычное значение "სხვა" в колонке Manufacturer.
Google определил как Грузинский язык. Перевод на английский other, что подходит по смыслу.
Какой шанс, что автор датасета Грузин?)

Удаляю строчки с სხვა в Manufacturer.

In [9]:
df = df[df["Manufacturer"] != "სხვა"]

Заменяю бинарные классификации колонками булевых типов:

In [10]:
df["Leather interior"] = pd.get_dummies(df["Leather interior"], drop_first=True)
df["Right_hand"] = pd.get_dummies(df["Wheel"], drop_first=True)
df = df.drop(["Wheel"], axis=1)

In [11]:
# конвертирую float в int
df = df.astype({"Cylinders": "int32"})

Функции для конвертации колонок в числовые типы: Mileage, Engine volume.
Функция для колонки Turbo.

In [12]:
def mileage_to_int(x: str) -> int:
    return int(x.replace(" km", ""))
df["Mileage"] = df["Mileage"].apply(mileage_to_int)


In [13]:
def has_turbo(x: str) -> bool:
    return "Turbo" in x
df["Turbo"] = df["Engine volume"].apply(has_turbo)

In [14]:
def engine_volume_to_float(x: str) -> float:
    return float(x.replace(" Turbo", ""))
df["Engine volume"] = df["Engine volume"].apply(engine_volume_to_float)

Колонки: Doors, Drive wheels, Gear box type привел к числовому формату.
Изменил имена колонок на более понятные и удалил Doors, Drive wheels, Gear.

In [15]:
df = pd.concat([
    df,
    pd.get_dummies(df["Doors"], drop_first=True),
    pd.get_dummies(df["Drive wheels"], drop_first=True),
    pd.get_dummies(df["Gear box type"], drop_first=True),
], axis=1)
df = df.rename(columns={"04-May": "4 doors", ">5": ">5 doors"})
df["Manufacturer and Model"] = df["Manufacturer"] + df["Model"]
df = df.drop(["Doors", "Drive wheels", "Gear box type", "Manufacturer", "Model"], axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(["Price", "ID"], axis=1), df["Price"], test_size=0.2, random_state=144, shuffle=True)

In [17]:
encoder_mm = LeaveOneOutEncoder()
X_train['Manufacturer and Model'] = encoder_mm.fit_transform(X_train['Manufacturer and Model'], y_train)

In [18]:
encoder_col = LeaveOneOutEncoder()
X_train['Color'] = encoder_col.fit_transform(X_train['Color'], y_train)

In [19]:
encoder_ft = LeaveOneOutEncoder()
X_train['Fuel type'] = encoder_ft.fit_transform(X_train['Fuel type'], y_train)

In [20]:
encoder_cat = LeaveOneOutEncoder()
X_train['Category'] = encoder_cat.fit_transform(X_train['Category'], y_train)

In [21]:
m = LinearRegression()
m.fit(X_train, y_train)

LinearRegression()

In [22]:
X_test['Manufacturer and Model'] = encoder_mm.transform(X_test['Manufacturer and Model'], y_test)
X_test['Color'] = encoder_col.transform(X_test['Color'], y_test)
X_test['Fuel type'] = encoder_ft.fit_transform(X_test['Fuel type'], y_test)
X_test['Category'] = encoder_cat.fit_transform(X_test['Category'], y_test)

In [23]:
prediction = m.predict(X_test)

In [24]:
mean_squared_error(y_test, prediction)

373411547.07397306